# A.4b RGB + Synthetic Depth 4-Channel with Reset BN

**Experiment:** A.4b - RGB+Synthetic Depth Fusion
**Input:** RGB + Synthetic Depth (4-channel RGBD)
**Objective:** Test fusion of RGB with synthetic depth (Depth-Anything-V2)
**Classes:** 1 (fresh_fruit_bunch)

## Key Features
1. **4-Channel Input**: RGB (3) + Synthetic Depth (1) = 4 channels
2. **Custom Trainer**: RGBD4ChTrainer with 4-channel support
3. **Monkey-Patch**: Modified imread for 4-channel image loading
4. **First Conv Adaptation**: Convert 3ch to 4ch input layer
5. **Reset BN**: Domain adaptation for RGBD input

## Prerequisites
- Upload RGB dataset as: `ffb-localization-dataset`
- Upload Synthetic Depth dataset as: `ffb-synthetic-depth`

## Uniform Augmentation (All Experiments)
- translate: 0.1
- scale: 0.5
- fliplr: 0.5
- hsv_h: 0.0 (disabled for uniformity)
- hsv_s: 0.0 (disabled for uniformity)
- hsv_v: 0.0 (disabled for uniformity)
- erasing: 0.0
- mosaic: 0.0
- mixup: 0.0

In [ ]:
# =============================================================================
# Cell 1: Setup & Install
# =============================================================================
!pip install -q ultralytics

import os
import sys
import torch
import torch.nn as nn
import numpy as np
import cv2
import shutil
import gc
import time
import json
from pathlib import Path
from datetime import datetime
from tqdm.auto import tqdm

os.environ["WANDB_DISABLED"] = "true"

# Auto-detect environment
IS_KAGGLE = os.path.exists('/kaggle/input') or os.path.exists('/kaggle')

if IS_KAGGLE:
    BASE_PATH = Path('/kaggle/working')
    RGB_DATASET = Path('/kaggle/input/ffb-localization-dataset/ffb_localization')
    DEPTH_DATASET = Path('/kaggle/input/ffb-synthetic-depth')
else:
    BASE_PATH = Path('D:/Work/Assisten Dosen/Anylabel/Experiments')
    RGB_DATASET = BASE_PATH / 'datasets' / 'ffb_localization'
    DEPTH_DATASET = BASE_PATH / 'datasets' / 'ffb_localization_depth_synthetic'

RUNS_PATH = BASE_PATH / 'runs' / 'detect'
RGBD_DATASET = BASE_PATH / 'rgbd_synthetic_4ch'
KAGGLE_OUTPUT = BASE_PATH / 'kaggleoutput'
KAGGLE_OUTPUT.mkdir(parents=True, exist_ok=True)

print("="*60)
print("A.4b RGB+SYNTHETIC DEPTH (4-CH) - ENVIRONMENT SETUP")
print("="*60)
print(f"Running on: {'Kaggle' if IS_KAGGLE else 'Local'}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    torch.cuda.empty_cache()
print(f"RGB Dataset: {RGB_DATASET}")
print(f"Depth Dataset: {DEPTH_DATASET}")
print(f"Output RGBD: {RGBD_DATASET}")
print("="*60)

In [ ]:
# =============================================================================
# Cell 2: Configuration AUGMENT_PARAMS
# =============================================================================
# Uniform augmentation parameters (consistent across all experiments)
AUGMENT_PARAMS = {
    'translate': 0.1,
    'scale': 0.5,
    'fliplr': 0.5,
    'hsv_h': 0.0,  # Disabled for uniformity
    'hsv_s': 0.0,  # Disabled for uniformity
    'hsv_v': 0.0,  # Disabled for uniformity
    'erasing': 0.0,
    'mosaic': 0.0,
    'mixup': 0.0,
    'degrees': 0.0,
    'copy_paste': 0.0,
}

# Training configuration
SEEDS = [42, 123, 456, 789, 101]
EXP_PREFIX = 'exp_a4b_rgbd_synth_v2'
EPOCHS = 100
PATIENCE = 30
IMGSZ = 640
BATCH_SIZE = 16
DEVICE = 0 if torch.cuda.is_available() else 'cpu'

print("="*60)
print("TRAINING CONFIGURATION")
print("="*60)
print(f"Experiment: A.4b RGB+Synthetic Depth (4-CH) (V2)")
print(f"Model: YOLOv11n")
print(f"Seeds: {SEEDS} ({len(SEEDS)} runs)")
print(f"Epochs: {EPOCHS} (patience: {PATIENCE})")
print(f"Image Size: {IMGSZ}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Device: {DEVICE}")
print("\nUniform Augmentation:")
for key, value in AUGMENT_PARAMS.items():
    print(f"  {key}: {value}")
print("\nSpecial Features:")
print("  - 4-Channel input (RGB+Synthetic Depth)")
print("  - Custom 4-channel trainer")
print("  - Reset BatchNorm for domain adaptation")
print("="*60)

In [ ]:
# =============================================================================
# Cell 3: Helper functions - convert_conv_to_4ch() and reset_bn_stats()
# =============================================================================
def convert_conv_to_4ch(conv_layer):
    """
    Convert a conv layer from 3ch to 4ch input.
    Copies RGB weights and initializes depth channel as mean of RGB.
    """
    if conv_layer.in_channels == 4:
        return conv_layer
    
    new_conv = nn.Conv2d(
        in_channels=4,
        out_channels=conv_layer.out_channels,
        kernel_size=conv_layer.kernel_size,
        stride=conv_layer.stride,
        padding=conv_layer.padding,
        bias=conv_layer.bias is not None
    )
    
    with torch.no_grad():
        new_conv.weight[:, :3, :, :] = conv_layer.weight.clone()
        new_conv.weight[:, 3:4, :, :] = conv_layer.weight.mean(dim=1, keepdim=True)
        if conv_layer.bias is not None:
            new_conv.bias = nn.Parameter(conv_layer.bias.clone())
    
    return new_conv


def reset_bn_stats(model, train_loader, num_batches=100, device='cuda'):
    """Reset running stats BatchNorm dengan 100 batch training data."""
    model.train()
    
    for module in model.modules():
        if isinstance(module, nn.BatchNorm2d):
            module.reset_running_stats()
            module.momentum = 0.1
    
    with torch.no_grad():
        for i, batch in enumerate(train_loader):
            if i >= num_batches:
                break
            imgs = batch['img'].to(device) if isinstance(batch, dict) else batch[0].to(device)
            _ = model(imgs)
    
    return model


def ensure_model_4ch(model):
    """Ensure model has 4-channel input."""
    try:
        if hasattr(model, 'model') and hasattr(model.model, 'model'):
            first_conv = model.model.model[0].conv
            if first_conv.in_channels == 3:
                print("[4ch] Converting model...")
                model.model.model[0].conv = convert_conv_to_4ch(first_conv)
                return True
        elif hasattr(model, 'model') and hasattr(model.model[0], 'conv'):
            first_conv = model.model[0].conv
            if first_conv.in_channels == 3:
                print("[4ch] Converting model...")
                model.model[0].conv = convert_conv_to_4ch(first_conv)
                return True
    except Exception as e:
        print(f"[4ch] Warning: {e}")
    return False

print("✓ Helper functions defined")

In [ ]:
# =============================================================================
# Cell 4: Create 4-Channel RGBD Dataset for Synthetic Depth
# =============================================================================
print("="*60)
print("CREATING 4-CHANNEL RGBD DATASET")
print("="*60)

for split in ['train', 'val', 'test']:
    print(f"\nProcessing {split.upper()}...")
    
    rgb_img_dir = RGB_DATASET / 'images' / split
    depth_img_dir = DEPTH_DATASET / 'images' / split
    rgb_lbl_dir = RGB_DATASET / 'labels' / split
    
    # Create output directories
    rgbd_img_dir = RGBD_DATASET / 'images' / split
    rgbd_lbl_dir = RGBD_DATASET / 'labels' / split
    rgbd_img_dir.mkdir(parents=True, exist_ok=True)
    rgbd_lbl_dir.mkdir(parents=True, exist_ok=True)
    
    # Get RGB files
    rgb_files = sorted(list(rgb_img_dir.glob('*.png')))
    
    for rgb_path in tqdm(rgb_files, desc=f"Creating 4-ch ({split})"):
        # Load RGB (3 channels)
        rgb = cv2.imread(str(rgb_path))
        if rgb is None:
            print(f"  Warning: Could not read {rgb_path}")
            continue
        
        # Load synthetic depth (1 channel)
        depth_path = depth_img_dir / rgb_path.name
        depth = cv2.imread(str(depth_path), cv2.IMREAD_GRAYSCALE)
        if depth is None:
            print(f"  Warning: Could not read {depth_path}")
            continue
        
        # Resize depth if needed
        if depth.shape[:2] != rgb.shape[:2]:
            depth = cv2.resize(depth, (rgb.shape[1], rgb.shape[0]))
        
        # Combine RGB (3) + Depth (1) = 4 channels
        depth_expanded = depth[:, :, np.newaxis]
        rgbd_4ch = np.concatenate([rgb, depth_expanded], axis=2)
        
        # Save 4-channel image
        output_img_path = rgbd_img_dir / rgb_path.name
        cv2.imwrite(str(output_img_path), rgbd_4ch)
    
    # Copy labels
    label_files = list(rgb_lbl_dir.glob('*.txt'))
    for lbl_path in label_files:
        shutil.copy(str(lbl_path), str(rgbd_lbl_dir / lbl_path.name))
    
    img_count = len(list(rgbd_img_dir.glob('*.png')))
    lbl_count = len(list(rgbd_lbl_dir.glob('*.txt')))
    print(f"  Done: {img_count} images, {lbl_count} labels")

# Verify sample
sample_img = list((RGBD_DATASET / 'images' / 'train').glob('*.png'))[0]
img = cv2.imread(str(sample_img), cv2.IMREAD_UNCHANGED)
print(f"\n✓ Sample image shape: {img.shape} (should be H x W x 4)")

In [ ]:
# =============================================================================
# Cell 5: Custom RGBD4ChTrainer and RGBD4ChValidator classes
# =============================================================================
import ultralytics.utils.patches as patches

_original_imread = patches.imread

def imread_4ch(filename, flags=cv2.IMREAD_UNCHANGED):
    """Always read with IMREAD_UNCHANGED to preserve 4 channels"""
    return _original_imread(filename, cv2.IMREAD_UNCHANGED)

patches.imread = imread_4ch
print("✓ Patched imread for 4-channel support")

from ultralytics import YOLO
from ultralytics.models.yolo.detect import DetectionTrainer, DetectionValidator
from ultralytics.nn.tasks import DetectionModel
import torch
print("✓ YOLO imported")


class RGBD4ChTrainer(DetectionTrainer):
    """
    Trainer khusus untuk 4-channel RGBD dengan BN reset menggunakan gambar training asli.
    Sesuai catatan dosen: forward pass 100 gambar training sebelum mulai training.
    """
    def __init__(self, overrides=None):
        super().__init__(overrides=overrides)  # Fix: explicit keyword argument
        # Register callback untuk BN reset setelah setup selesai
        self.add_callback("on_train_start", self._bn_reset_callback)
        self._bn_reset_done = False
    
    def get_model(self, cfg=None, weights=None, verbose=True):
        """Load model dan modifikasi first conv layer untuk 4 channels."""
        model = super().get_model(cfg, weights, verbose)

        # Modifikasi first conv layer dari 3ch ke 4ch
        first_conv = model.model[0].conv
        if first_conv.in_channels == 3:
            print("[Trainer] Converting first conv: 3ch → 4ch...")
            model.model[0].conv = convert_conv_to_4ch(first_conv)
            print(f"[Trainer] ✅ Converted! Shape: {model.model[0].conv.weight.shape}")
        else:
            print("[Trainer] Model sudah 4-channel")

        return model
    
    def build_dataset(self, img_path, mode="train", batch=None):
        """Override: gunakan RGBDDataset yang load 4 channel."""
        from ultralytics.data.dataset import YOLODataset
        import cv2
        import numpy as np
        from pathlib import Path
        
        class RGBDDataset(YOLODataset):
            """Dataset yang load 4-channel RGBD images."""
            def load_image(self, i):
                """Load 4-channel image."""
                im = cv2.imread(self.im_files[i], cv2.IMREAD_UNCHANGED)
                if im is None:
                    raise FileNotFoundError(f"Image Not Found {self.im_files[i]}")
                
                # Ensure 4 channels
                if len(im.shape) == 2:
                    im = cv2.cvtColor(im, cv2.COLOR_GRAY2BGR)
                    im = np.dstack([im, im[:,:,0]])
                elif im.shape[2] == 3:
                    depth_channel = np.zeros((im.shape[0], im.shape[1], 1), dtype=np.uint8)
                    im = np.dstack([im, depth_channel])
                
                h0, w0 = im.shape[:2]
                r = self.imgsz / max(h0, w0)
                if r != 1:
                    interp = cv2.INTER_LINEAR if r > 1 else cv2.INTER_AREA
                    im = cv2.resize(im, (int(w0 * r), int(h0 * r)), interpolation=interp)
                
                return im, (h0, w0), im.shape[:2]
        
        return RGBDDataset(
            img_path=img_path,
            imgsz=self.args.imgsz,
            batch_size=batch,
            augment=mode == "train",
            hyp=self.args,
            rect=mode == "val",
            cache=self.args.cache,
            single_cls=self.args.single_cls,
            stride=max(self.model.stride) if hasattr(self.model, 'stride') else 32,
            pad=0.0 if mode == "train" else 0.5,
            prefix=f"{mode}: ",
        )
    
    def _bn_reset_callback(self, trainer):
        """
        Callback: Reset BatchNorm dengan forward pass 100 gambar training.
        Dipanggil otomatis saat on_train_start.
        """
        if self._bn_reset_done:
            return
            
        print(f"\n[BN Reset] Resetting BatchNorm dengan 100 gambar training...")
        
        # 1. Reset running stats
        bn_layers = []
        for module in self.model.modules():
            if isinstance(module, nn.BatchNorm2d):
                module.reset_running_stats()
                module.momentum = 0.1
                bn_layers.append(module)
        print(f"[BN Reset] Found {len(bn_layers)} BatchNorm layers")
        
        # 2. Set model to train mode
        self.model.train()
        
        # 3. Forward pass dengan 100 gambar training asli
        device = next(self.model.parameters()).device
        images_processed = 0
        n_images = 100
        
        with torch.no_grad():
            for batch in self.train_loader:
                if images_processed >= n_images:
                    break
                
                images = batch['img'].to(device, non_blocking=True)
                print(f"[BN Reset] Batch shape: {images.shape}, dtype: {images.dtype}")
                
                # Forward pass - update BN running stats
                _ = self.model(images)
                
                images_processed += len(images)
        
        print(f"[BN Reset] ✅ Completed forward pass pada {images_processed} gambar")
        
        # 4. Verify BN stats
        if bn_layers:
            sample_bn = bn_layers[0]
            print(f"[BN Reset] Sample BN running_mean: [{sample_bn.running_mean.min():.4f}, {sample_bn.running_mean.max():.4f}]")
        
        self._bn_reset_done = True

    def get_validator(self):
        self.loss_names = "box_loss", "cls_loss", "dfl_loss"
        return RGBD4ChValidator(
            self.test_loader,
            save_dir=self.save_dir,
            args=self.args,
            _callbacks=self.callbacks
        )


class RGBD4ChValidator(DetectionValidator):
    """Validator yang convert model ke 4ch dan pakai RGBDDataset."""
    
    def build_dataset(self, img_path, mode="val", batch=None):
        """Override: gunakan RGBDDataset yang load 4 channel."""
        from ultralytics.data.dataset import YOLODataset
        import cv2
        import numpy as np
        
        class RGBDDataset(YOLODataset):
            """Dataset yang load 4-channel RGBD images."""
            def load_image(self, i):
                """Load 4-channel image."""
                im = cv2.imread(self.im_files[i], cv2.IMREAD_UNCHANGED)
                if im is None:
                    raise FileNotFoundError(f"Image Not Found {self.im_files[i]}")
                
                # Ensure 4 channels
                if len(im.shape) == 2:
                    im = cv2.cvtColor(im, cv2.COLOR_GRAY2BGR)
                    im = np.dstack([im, im[:,:,0]])
                elif im.shape[2] == 3:
                    depth_channel = np.zeros((im.shape[0], im.shape[1], 1), dtype=np.uint8)
                    im = np.dstack([im, depth_channel])
                
                h0, w0 = im.shape[:2]
                r = self.imgsz / max(h0, w0)
                if r != 1:
                    interp = cv2.INTER_LINEAR if r > 1 else cv2.INTER_AREA
                    im = cv2.resize(im, (int(w0 * r), int(h0 * r)), interpolation=interp)
                
                return im, (h0, w0), im.shape[:2]
        
        return RGBDDataset(
            img_path=img_path,
            imgsz=self.args.imgsz,
            batch_size=batch,
            augment=False,
            hyp=self.args,
            rect=True,
            cache=self.args.cache if hasattr(self.args, 'cache') else False,
            single_cls=self.args.single_cls if hasattr(self.args, 'single_cls') else False,
            stride=int(self.stride) if hasattr(self, 'stride') else 32,
            pad=0.5,
            prefix=f"{mode}: ",
        )
    
    def setup_model(self):
        super().setup_model()
        if self.model is not None:
            ensure_model_4ch(self.model)
            print("[Validator] Model 4ch ready")


print("✅ Custom Trainer & Validator defined")
print("   - Auto-convert 3ch → 4ch di get_model()")
print("   - build_dataset() override untuk 4-channel RGBDDataset")
print("   - BN reset dengan 100 gambar training asli via callback on_train_start")

In [ ]:
# =============================================================================
# Cell 6: Create 4ch dataset and YAML
# =============================================================================
yaml_content = f"""
# A.4b RGB+Synthetic Depth 4-Channel Dataset Configuration
path: {RGBD_DATASET}
train: images/train
val: images/val
test: images/test

nc: 1
channels: 4

names:
  0: fresh_fruit_bunch
"""

config_path = RGBD_DATASET / 'dataset_rgbd_synthetic_v2.yaml'
with open(config_path, 'w') as f:
    f.write(yaml_content)

print(f"✅ Config saved: {config_path}")
print("\nConfig contents:")
print("-"*40)
print(yaml_content)

In [ ]:
# =============================================================================
# Cell 7: Training loop (Simplified - BN reset handled in trainer)
# =============================================================================
results_all = {}
training_times = {}

print("\n" + "="*60)
print("STARTING TRAINING LOOP")
print("="*60)

for idx, seed in enumerate(SEEDS, 1):
    start_time = time.time()
    
    print(f"\n{'='*60}")
    print(f"TRAINING A.4b RGBD SYNTHETIC - Seed {seed} ({idx}/{len(SEEDS)})")
    print(f"{'='*60}")
    
    # Set seeds
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    
    try:
        # Create trainer - 4ch conversion dan BN reset otomatis di get_model()
        trainer = RGBD4ChTrainer(overrides={
            'model': 'yolo11n.pt',
            'data': str(config_path),
            'imgsz': IMGSZ,
            'epochs': EPOCHS,
            'batch': BATCH_SIZE,
            'device': DEVICE,
            'seed': seed,
            'name': f"{EXP_PREFIX}_seed{seed}",
            'project': str(RUNS_PATH),
            'exist_ok': True,
            'pretrained': True,
            'patience': PATIENCE,
            'val': True,
            **AUGMENT_PARAMS,
        })
        
        # Train - conversion 4ch dan BN reset sudah dilakukan di get_model()
        trainer.train()
        
        elapsed = time.time() - start_time
        training_times[seed] = elapsed
        
        results_all[seed] = {
            'model_path': str(RUNS_PATH / f"{EXP_PREFIX}_seed{seed}" / "weights" / "best.pt"),
            'epochs_trained': EPOCHS,
            'completed': True,
        }
        
        print(f"\n✓ Seed {seed} completed!")
        print(f"  Time: {elapsed/60:.1f} minutes")
        
    except Exception as e:
        print(f"\n✗ Seed {seed} failed: {e}")
        import traceback
        traceback.print_exc()
        results_all[seed] = {'error': str(e), 'completed': False}
    
    # Cleanup
    if 'trainer' in locals():
        del trainer
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

print("\n" + "="*60)
print("TRAINING LOOP COMPLETED")
print("="*60)

# Print summary
print("\n📊 RESULTS SUMMARY:")
successful = sum(1 for r in results_all.values() if r.get('completed', False))
print(f"Successful: {successful}/{len(SEEDS)}")
for seed, res in results_all.items():
    if res.get('completed', False):
        print(f"  Seed {seed}: ✓ Completed")
    else:
        print(f"  Seed {seed}: ✗ FAILED - {res.get('error', 'Unknown')[:50]}...")

In [ ]:
# =============================================================================
# Cell 8: Evaluation
# =============================================================================
results_dict = {}

print("\n" + "="*60)
print("EVALUATION ON TEST SET")
print("="*60)

for seed in SEEDS:
    model_path = RUNS_PATH / f"{EXP_PREFIX}_seed{seed}" / "weights" / "best.pt"
    
    if not model_path.exists():
        print(f"Model not found: {model_path}")
        continue
    
    print(f"\nSeed {seed}:")
    
    try:
        model = YOLO(str(model_path))
        
        # Convert to 4ch if needed
        first_conv = model.model.model[0].conv
        if first_conv.in_channels == 3:
            print("  Converting to 4ch...")
            model.model.model[0].conv = convert_conv_to_4ch(first_conv)
        
        # Verify 4-channel
        print(f"  Model input channels: {model.model.model[0].conv.in_channels}")
        
        metrics = model.val(
            data=str(config_path),
            split="test",
            device=DEVICE,
            name=f"test_{EXP_PREFIX}_seed{seed}",
            exist_ok=True,
        )
        
        results_dict[seed] = {
            'mAP50': metrics.box.map50,
            'mAP50-95': metrics.box.map,
            'Precision': metrics.box.mp,
            'Recall': metrics.box.mr,
        }
        
        print(f"  mAP50: {metrics.box.map50:.4f}, mAP50-95: {metrics.box.map:.4f}")
        
        del model
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            
    except Exception as e:
        print(f"  Evaluation failed: {e}")

print("\n" + "="*60)
print("EVALUATION COMPLETED")
print("="*60)

In [ ]:
# =============================================================================
# Cell 9: Summary
# =============================================================================
import pandas as pd

if results_dict:
    df = pd.DataFrame(results_dict).T
    df.index.name = 'Seed'
    
    avg = df.mean()
    std = df.std()
    min_vals = df.min()
    max_vals = df.max()
    
    print("\n" + "="*60)
    print("A.4b RGB+SYNTHETIC DEPTH (4-CH) (V2) - FINAL RESULTS")
    print("="*60 + "\n")
    print(df.to_string(float_format=lambda x: f"{x:.4f}"))
    
    print("\n" + "-"*60)
    print("STATISTICAL SUMMARY")
    print("-"*60)
    print(f"{'Metric':<15} {'Mean':>10} {'Std':>10} {'Min':>10} {'Max':>10}")
    print("-"*60)
    for col in df.columns:
        print(f"{col:<15} {avg[col]:>10.4f} {std[col]:>10.4f} {min_vals[col]:>10.4f} {max_vals[col]:>10.4f}")
    
    best_seed = df['mAP50'].idxmax()
    print(f"\n✓ Best Seed: {best_seed} (mAP50: {df.loc[best_seed, 'mAP50']:.4f})")
    
    print("="*60)
else:
    print("No results to display.")

In [ ]:
# =============================================================================
# Cell 10: Save results
# =============================================================================
output_file = KAGGLE_OUTPUT / 'a4b_rgbd_synthetic_v2_results.txt'

with open(output_file, 'w') as f:
    f.write("="*60 + "\n")
    f.write("A.4b RGB+Synthetic Depth (4-CH) (V2) Results\n")
    f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Environment: {'Kaggle' if IS_KAGGLE else 'Local'}\n")
    f.write("="*60 + "\n\n")
    
    f.write("Configuration:\n")
    f.write(f"  Model: YOLOv11n\n")
    f.write(f"  Input: 4-Channel (RGB+Synthetic Depth)\n")
    f.write(f"  Depth Source: Depth-Anything-V2\n")
    f.write(f"  Epochs: {EPOCHS} (patience: {PATIENCE})\n")
    f.write(f"  Image Size: {IMGSZ}\n")
    f.write(f"  Batch Size: {BATCH_SIZE}\n")
    f.write(f"  Seeds: {SEEDS}\n")
    f.write("\nUniform Augmentation:\n")
    for key, value in AUGMENT_PARAMS.items():
        f.write(f"  {key}: {value}\n")
    f.write("\nSpecial Features:\n")
    f.write("  - 4-Channel input (RGB+Synthetic Depth)\n")
    f.write("  - Custom RGBD4ChTrainer\n")
    f.write("  - Reset BatchNorm for domain adaptation\n")
    
    if results_dict:
        f.write("\n" + "="*60 + "\n")
        f.write("Per-Seed Results:\n")
        f.write("="*60 + "\n")
        f.write(df.to_string(float_format=lambda x: f"{x:.4f}"))
        
        f.write("\n\n" + "-"*60 + "\n")
        f.write("Summary (Mean ± Std):\n")
        f.write("-"*60 + "\n")
        for col in df.columns:
            f.write(f"  {col}: {avg[col]:.4f} ± {std[col]:.4f}\n")
        
        f.write(f"\nBest Seed: {best_seed}\n")

print(f"\n✓ Results saved: {output_file}")

# JSON output
json_output = {
    'experiment': 'A.4b',
    'variant': 'V2',
    'seeds': SEEDS,
    'config': {
        'model': 'yolo11n',
        'input_channels': 4,
        'depth_source': 'synthetic',
        'epochs': EPOCHS,
        'patience': PATIENCE,
        'imgsz': IMGSZ,
        'batch': BATCH_SIZE,
        'augmentation': AUGMENT_PARAMS,
        'reset_bn': True,
    },
    'results': {str(k): v for k, v in results_dict.items()},
    'summary': {
        'mean': {k: float(v) for k, v in avg.items()},
        'std': {k: float(v) for k, v in std.items()},
        'best_seed': int(best_seed) if results_dict else None,
    } if results_dict else None,
}

json_file = KAGGLE_OUTPUT / 'a4b_rgbd_synthetic_v2_results.json'
with open(json_file, 'w') as f:
    json.dump(json_output, f, indent=2)

print(f"✓ JSON saved: {json_file}")

In [ ]:
# =============================================================================
# Cell 11: Create archives
# =============================================================================
print("\n" + "="*60)
print("CREATING ARCHIVES")
print("="*60 + "\n")

if RUNS_PATH.exists():
    runs_zip = BASE_PATH / 'a4b_rgbd_synthetic_v2_runs.zip'
    shutil.make_archive(str(runs_zip.with_suffix('')), 'zip', RUNS_PATH)
    size_mb = runs_zip.stat().st_size / 1024 / 1024
    print(f"✓ a4b_rgbd_synthetic_v2_runs.zip: {size_mb:.1f} MB")

output_zip = BASE_PATH / 'a4b_rgbd_synthetic_v2_output.zip'
shutil.make_archive(str(output_zip.with_suffix('')), 'zip', KAGGLE_OUTPUT)
size_mb = output_zip.stat().st_size / 1024 / 1024
print(f"✓ a4b_rgbd_synthetic_v2_output.zip: {size_mb:.1f} MB")

print("\n" + "="*60)
print("ALL DONE!")
print("="*60)
print("\nDownload from Output tab:")
print("  - a4b_rgbd_synthetic_v2_runs.zip (training runs)")
print("  - a4b_rgbd_synthetic_v2_output.zip (results)")